In [ ]:
import tweepy
import pandas as pd
import numpy as np
import time
import re
import gensim
import nltk
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from gensim.utils import simple_preprocess
from nltk.tokenize.treebank import TreebankWordDetokenizer

In [ ]:
# 크롤링에 필요한 조건:
# 트위터 디벨로퍼 계정 신청 및 승인,
# 승인까지 완료하면 아래 사항 발급받아서 저장해두기
consumer_key = "XXXXXXXXX"
consumer_secret = "XXXXXXXXX"
access_token = "XXXXXXXXX"
access_token_secret = "XXXXXXXXX"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [ ]:
consumer_key = "5Z2y8IXKj8HN1bZ3H7RzBUZ4o"
consumer_secret = "NSEXPSxm7dcBqealZP5HGateYlwWbv5OC5M8LMrkyrjgLzHJPm"
access_token = "1389110629030531074-JwTuiyDfjV9Mb6dzJJi5pizxz9JEB0"
access_token_secret = "usD8JJ9ruuAvGAvlN3scUDxRuWMey7qEg2ag1YPQmCvlp"
# bearer_token = "AAAAAAAAAAAAAAAAAAAAAPT%2FPAEAAAAAtmvrT%2BneFJs2JdOSPtaW1ro7vTI%3Df6C0ie40Rou9VChOWZoedKbf27atFLOT1y80uFVGgsWCyIkenq"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [ ]:
df = pd.read_csv('appliances_no_null_values.csv', index_col=False)
df.head()

,rating,reviewer_id,product_id,unix_review_time,category,description,also_buy,brand,main_cat,price,review_text,rating_class,time
0,5,A2OXDRWBASV91Y,B00004SQHD,1366416000,"['Appliances', 'Parts & Accessories', 'Range P...",['Coleman Cable 09045 5-Foot Range Cord Made o...,[],Coleman Cable,Tools & Home Improvement,$1.41,complete package I like the fact that the wire...,good,2013-04-20
1,5,A2KG6AWJSWILPR,B00004SQHD,1363392000,"['Appliances', 'Parts & Accessories', 'Range P...",['Coleman Cable 09045 5-Foot Range Cord Made o...,[],Coleman Cable,Tools & Home Improvement,$1.41,Perfect Fit Needed another couple of feet with...,good,2013-03-16
2,5,A2CBE6VYOARZN4,B00004SQHD,1487030400,"['Appliances', 'Parts & Accessories', 'Range P...",['Coleman Cable 09045 5-Foot Range Cord Made o...,[],Coleman Cable,Tools & Home Improvement,$1.41,Everything works Good quality at a reasonable ...,good,2017-02-14
3,5,AVKOTZD5ZIOX5,B00004SQHD,1482278400,"['Appliances', 'Parts & Accessories', 'Range P...",['Coleman Cable 09045 5-Foot Range Cord Made o...,[],Coleman Cable,Tools & Home Improvement,$1.41,50 amp cable Planning on hooking this to my RV,good,2016-12-21
4,5,A33DUVUIC7G553,B00004SQHD,1460678400,"['Appliances', 'Parts & Accessories', 'Range P...",['Coleman Cable 09045 5-Foot Range Cord Made o...,[],Coleman Cable,Tools & Home Improvement,$1.41,"Good range cord, standard nema 14-50p fits wit...",good,2016-04-15


In [ ]:
def recentquery500(prod_id):
    text_query = df[df['product_id'] == prod_id]['brand'][0]
    count = 500
    try:
        tweets = tweepy.Cursor(api.search, q=text_query).items(count)
        tweets_list = [[tweet.created_at, tweet.id, tweet.text] for tweet in tweets]
        tweets_df = pd.DataFrame(tweets_list)
    except BaseException as e:
        print('failed on_status,',str(e))
        time.sleep(3)
    return tweets_df

In [ ]:
# 최근 30일간의 트윗 중에서 상품의 브랜드명을 포함하는 모든 트윗을 검색 및 출력, 최대 500개까지

In [ ]:
# 결과 데이터프레임에서 감성분석에 사용할 텍스트만 array로 출력하기
result = recentquery500('B00004SQHD')[2].values
result

array(['COLEMAN WIRE AND CABLE NIGERIA 🇳🇬\n95MM 4CORE ARMOURED CABLE NIGERIA\n27000\n08096654860\n07041480020\n08176783390 https://t.co/9dVwAAH0Cz',
       'DIVINE SOLACE ELECTRICAL ENTERPRISES\nDEALER in all kinds of electrical equipment\nDistributor Coleman wire and cable… https://t.co/CzoMugcdBo',
       'RT @davemeltzerWON: In the last seven days, Raw was No. 1 on Monday on cable, AEW was No. 1 on Wednesday on cable, Smackdown was No. 1 on n…',
       'RT @WalshFreedom: He got himself vaccinated as soon as he could.\n\nBut every night on his #1 rated cable news show, he scares the shit out o…'],
      dtype=object)

In [ ]:
def depure_data(data):
    
    #Removing URLs with a regular expression
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    data = url_pattern.sub(r'', data)

    # Remove Emails
    data = re.sub('\S*@\S*\s?', '', data)

    # Remove new line characters
    data = re.sub('\s+', ' ', data)

    # Remove distracting single quotes
    data = re.sub("\'", "", data)
        
    return data

In [ ]:
temp = []
#Splitting pd.Series to list
data_to_list = result.tolist()
for i in range(len(data_to_list)):
    temp.append(depure_data(data_to_list[i]))
list(temp[:5])

['COLEMAN WIRE AND CABLE NIGERIA 🇳🇬 95MM 4CORE ARMOURED CABLE NIGERIA 27000 08096654860 07041480020 08176783390 ',
 'DIVINE SOLACE ELECTRICAL ENTERPRISES DEALER in all kinds of electrical equipment Distributor Coleman wire and cable… ',
 'RT In the last seven days, Raw was No. 1 on Monday on cable, AEW was No. 1 on Wednesday on cable, Smackdown was No. 1 on n…',
 'RT He got himself vaccinated as soon as he could. But every night on his #1 rated cable news show, he scares the shit out o…']

In [ ]:
train = pd.read_csv('appliances_keras_sentiment.csv')
train = train.sort_values(by='rating_class')
train = train[:120000]

In [ ]:
def depure_data(data):
    
    #Removing URLs with a regular expression
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    data = url_pattern.sub(r'', data)

    # Remove Emails
    data = re.sub('\S*@\S*\s?', '', data)

    # Remove new line characters
    data = re.sub('\s+', ' ', data)

    # Remove distracting single quotes
    data = re.sub("\'", "", data)
        
    return data

In [ ]:
temp = []
#Splitting pd.Series to list
data_to_list = train['review_text'].values.tolist()
for i in range(len(data_to_list)):
    temp.append(depure_data(data_to_list[i]))
list(temp[:5])

['One Star Housing came broken',
 'Not helpful for "traveling" washing machine These did not help our "traveling" washer at all, which vibrates so hard in the spin cycle it moves around the floor. Vibrated right off the pads.',
 'doesnt work. Does not stick to granites rough surface and it needs a smaller profile, does not allow the dishwasher door to shut. Disappointed.',
 'Jenn-Air knob No problem with the purchase. However, the knobs are cheap plastic. I dont understand why they are not aluminum or some other metal. The price is good enough, but the postage is ridiculously high. Jenn-Air, when contacted, showed no interest in considering suggestions related to the poor quality of the knobs. I would suggest buying another brand in the future.',
 'One Star I returned the item, it didnt fit my dryer.']

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        

data_words = list(sent_to_words(temp))

In [ ]:
def detokenize(text):
    return TreebankWordDetokenizer().detokenize(text)

In [ ]:
data = []
for i in range(len(data_words)):
    data.append(detokenize(data_words[i]))

In [ ]:
data = np.array(data)

In [ ]:
max_words = 5000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data)
best_model = keras.models.load_model("best_model0.hdf5")

In [ ]:
def sentiment_predict(new_sentence):
    encoded = tokenizer.texts_to_sequences([new_sentence])    # 정수 인코딩
    pad_new = pad_sequences(encoded, maxlen = max_len)     # 패딩
    scores = best_model.predict(pad_new)   # 예측
    score = scores[0, 1]
    if score > 0.5:
        print(f'해당 문장({new_sentence[:30]}...)은 {score*100}% 확률로 긍정 리뷰입니다.')
    else:
        print(f'해당 문장({new_sentence[:30]}...)은 {(1-score)*100}% 확률로 부정 리뷰입니다.')

In [ ]:
result

array(['COLEMAN WIRE AND CABLE NIGERIA 🇳🇬\n95MM 4CORE ARMOURED CABLE NIGERIA\n27000\n08096654860\n07041480020\n08176783390 https://t.co/9dVwAAH0Cz',
       'DIVINE SOLACE ELECTRICAL ENTERPRISES\nDEALER in all kinds of electrical equipment\nDistributor Coleman wire and cable… https://t.co/CzoMugcdBo',
       'RT @davemeltzerWON: In the last seven days, Raw was No. 1 on Monday on cable, AEW was No. 1 on Wednesday on cable, Smackdown was No. 1 on n…',
       'RT @WalshFreedom: He got himself vaccinated as soon as he could.\n\nBut every night on his #1 rated cable news show, he scares the shit out o…'],
      dtype=object)

In [ ]:
for num in range(len(result)):
    sentence = result[num]
    sentiment_predict(sentence)

해당 문장(COLEMAN WIRE AND CABLE NIGERIA...)은 61.50926947593689% 확률로 긍정 리뷰입니다.
해당 문장(DIVINE SOLACE ELECTRICAL ENTER...)은 50.90205669403076% 확률로 긍정 리뷰입니다.
해당 문장(RT @davemeltzerWON: In the las...)은 64.75542187690735% 확률로 긍정 리뷰입니다.
해당 문장(RT @WalshFreedom: He got himse...)은 62.37553358078003% 확률로 부정 리뷰입니다.
